In [21]:
!pip install cassandra-driver

In [22]:
import json
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Load credentials from JSON file
with open("cassandra-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

# Connect to Cassandra
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cloud_config = {"secure_connect_bundle": "secure-connect-cassandra.zip"}
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Set keyspace
session.set_keyspace('assignment')

# Confirm connection
if session:
    print("Connected!")
else:
    print("Connection failed.")


Connected!


In [23]:
# Bronze Table
session.execute("""
CREATE TABLE IF NOT EXISTS bronze_sales (
    id UUID PRIMARY KEY,
    region TEXT,
    product TEXT,
    customer TEXT,
    total_revenue FLOAT,
    total_profit FLOAT
);
""")

# Silver Table
session.execute("""
CREATE TABLE IF NOT EXISTS silver_sales (
    id UUID PRIMARY KEY,
    region TEXT,
    product TEXT,
    customer TEXT,
    total_revenue FLOAT,
    total_profit FLOAT
);
""")

# Gold Tables
session.execute("""
CREATE TABLE IF NOT EXISTS gold_sales_summary (
    region TEXT PRIMARY KEY,
    total_sales FLOAT
);
""")

session.execute("""
CREATE TABLE IF NOT EXISTS gold_top_customers (
    customer TEXT PRIMARY KEY,
    total_revenue FLOAT
);
""")

session.execute("""
CREATE TABLE IF NOT EXISTS gold_product_performance (
    product TEXT PRIMARY KEY,
    total_sales FLOAT
);
""")

print("Tables created successfully.")


Tables created successfully.


In [24]:
import pandas as pd
import uuid

# Load the raw data from the CSV
file_path = "sales_100.csv"  # Ensure you upload this file to Colab
bronze_data = pd.read_csv(file_path)

print("Data loaded successfully.")
bronze_data.head()


Data loaded successfully.


,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,UnitsSold,UnitPrice,UnitCost,TotalRevenue,TotalCost,TotalProfit
0,Sub-Saharan Africa,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.33,6.92,14862.69,11023.56,3839.13
1,Middle East and North Africa,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.28,35.84,503890.08,165258.24,338631.84
2,Australia and Oceania,Papua New Guinea,Meat,Offline,M,5/15/2015,940995585,6/4/2015,360,421.89,364.69,151880.40,131288.40,20592.00
3,Sub-Saharan Africa,Djibouti,Clothes,Offline,H,5/17/2017,880811536,7/2/2017,562,109.28,35.84,61415.36,20142.08,41273.28
4,Europe,Slovakia,Beverages,Offline,L,10/26/2016,174590194,12/4/2016,3973,47.45,31.79,188518.85,126301.67,62217.18


In [25]:
# Insert raw data into Bronze Table
for _, row in bronze_data.iterrows():
    query = f"""
    INSERT INTO bronze_sales (id, region, product, customer, total_revenue, total_profit)
    VALUES ({uuid.uuid4()}, '{row['Region']}', '{row['Item Type']}', '{row['Country']}', {row['TotalRevenue']}, {row['TotalProfit']});
    """
    session.execute(query)

print("Data inserted into Bronze table.")


Data inserted into Bronze table.


In [26]:
# Clean data (remove duplicates and null values)
silver_data = bronze_data.dropna().drop_duplicates()

# Insert cleaned data into Silver Table
for _, row in silver_data.iterrows():
    query = f"""
    INSERT INTO silver_sales (id, region, product, customer, total_revenue, total_profit)
    VALUES ({uuid.uuid4()}, '{row['Region']}', '{row['Item Type']}', '{row['Country']}', {row['TotalRevenue']}, {row['TotalProfit']});
    """
    session.execute(query)

print("Data inserted into Silver table.")


Data inserted into Silver table.


In [27]:
# Group by Region
sales_by_region = silver_data.groupby('Region')['TotalRevenue'].sum().reset_index()

# Insert into Gold Sales Summary Table
for _, row in sales_by_region.iterrows():
    query = f"""
    INSERT INTO gold_sales_summary (region, total_sales)
    VALUES ('{row['Region']}', {row['TotalRevenue']});
    """
    session.execute(query)

print("Data inserted into Gold Sales Summary table.")


Data inserted into Gold Sales Summary table.


In [28]:
# Group by Customer
top_customers = silver_data.groupby('Country')['TotalRevenue'].sum().nlargest(10).reset_index()

# Insert into Gold Top Customers Table
for _, row in top_customers.iterrows():
    query = f"""
    INSERT INTO gold_top_customers (customer, total_revenue)
    VALUES ('{row['Country']}', {row['TotalRevenue']});
    """
    session.execute(query)

print("Data inserted into Gold Top Customers table.")


Data inserted into Gold Top Customers table.


In [29]:
# Group by Product
product_performance = silver_data.groupby('Item Type')['TotalRevenue'].sum().reset_index()

# Insert into Gold Product Performance Table
for _, row in product_performance.iterrows():
    query = f"""
    INSERT INTO gold_product_performance (product, total_sales)
    VALUES ('{row['Item Type']}', {row['TotalRevenue']});
    """
    session.execute(query)

print("Data inserted into Gold Product Performance table.")


Data inserted into Gold Product Performance table.


In [30]:
# Query Gold Table Data
rows = session.execute("SELECT * FROM 'gold_sales_summary';")
print("Gold Sales Summary:")
for row in rows:
    print(row)

rows = session.execute("SELECT * FROM gold_top_customers;")
print("Gold Top Customers:")
for row in rows:
    print(row)

rows = session.execute("SELECT * FROM gold_product_performance;")
print("Gold Product Performance:")
for row in rows:
    print(row)


Gold Sales Summary:
Row(region='Australia and Oceania', total_sales=10711258.0)
Row(region='Europe', total_sales=34964748.0)
Row(region='Middle East and North Africa', total_sales=24765128.0)
Row(region='Central America and the Caribbean', total_sales=17570836.0)
Row(region='Asia', total_sales=28840812.0)
Row(region='Sub-Saharan Africa', total_sales=24225438.0)
Row(region='North America', total_sales=3611757.5)
Gold Top Customers:
Row(customer='Nicaragua', total_revenue=5944506.0)
Row(customer='Estonia', total_revenue=6666661.5)
Row(customer='Dominica', total_revenue=5006276.5)
Row(customer='Antigua and Barbuda ', total_revenue=4100669.25)
Row(customer='Algeria', total_revenue=4227287.0)
Row(customer='Finland', total_revenue=5552598.0)
Row(customer='Pakistan', total_revenue=4205821.5)
Row(customer='United Arab Emirates', total_revenue=6253569.5)
Row(customer='Japan', total_revenue=5608790.0)
Row(customer='Zimbabwe', total_revenue=6266594.0)
Gold Product Performance:
Row(product='Househ